<a href="https://colab.research.google.com/github/hexabenja/DataAnalysisScraper/blob/main/DataWebScr4per.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Esta celda instala las bibliotecas necesarias para el web scraping con Langchain, incluidas langchain-community y beautifulsoup4.

In [ ]:
%pip install -qU langchain-community beautifulsoup4

Esta celda importa bs4 y WebBaseLoader para cargar contenido de una página web específica (En este ejemplo catálogo CREST de sodimac).

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

 Inicializa el loader con la URL y obtiene el documento de forma asíncrona.

In [ ]:
page_url = "https://www.sodimac.cl/sodimac-cl/marca/CREST"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]